# Notebook 1 - Exploring causal graphs and relationships (student notebook)

In this notebook we'll work through some basic examples of causal relationships, in the form of causal graphs.

__NOTE FOR WINDOWS USERS__: You may need to run the following lines in order for `graphviz` to work in this notebook:

```
import os
os.environ["PATH"] += os.pathsep + 'C:\Program Files\Graphviz\bin\' # Or to whatever folder on your machine has the graphviz bin files
```

In [ ]:
from causalgraphicalmodels.csm import StructuralCausalModel, linear_model, logistic_model
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm, pearsonr

%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 5]

In [ ]:
def clean_corr(corr_results):
    """
    Takes the output of the scipy.stats.pearsonr function (Pearson correlation) and cleans it up for easy viewing
    
    Args:
        corr_results: Tuple of floats, the output from scipy.stats.pearsonr
        
    Returns: None
    """
    print(f"r = {round(corr_results[0], 3)} (p =  {round(corr_results[1], 3)})")

## Classic confounding

Let's begin by creating a simple causal model for confounding. Remember, a covariate confounds a relationship between two variables of interest when it causes both. This can induce a spurious association between these two variables if none truly exists.

In [ ]:
# Create a toy model here for confounding. We can specify the relationships between three variables
cie_example = StructuralCausalModel({
    "smoking": lambda n_samples: np.random.binomial(1, p=0.30, size=n_samples), 
    "coffee": linear_model(["smoking"], [1.5]),
    "lung cancer risk": linear_model(["smoking"], [6])
})

In [ ]:
cie_cgm = cie_example.cgm
cie_cgm.draw()

Now that we've specified the relationships between these variables, we can simulate a dataset with these relationships...

In [ ]:
data = cie_example.sample(n_samples=100000)
data.head()

A statistical reminder: Pearson correlation coefficients can vary between -1 to 1. -1 indicates a perfect negative/inverse correlation between two variables, while 0 indicates no correlation, and 1 indicates a perfect positive correlation. We can see that there is an apparent moderately strong, positive association between the coffee and lung cancer risk variables

In [ ]:
clean_corr(
    pearsonr(data['coffee'], data['lung cancer risk'])
)

As a sanity check, let's plot the relationship between coffee and lung cancer risk

In [ ]:
plt.scatter(data['coffee'], data['lung cancer risk'])

There's a lot of overlapping data there, so let's create a hexbin heatmap plot to better see what's going on here.

In [ ]:
plt.hexbin(data['coffee'], data['lung cancer risk'], gridsize=50)

We can disrupt the confounding by stratifying on smoking. Let's filter the dataset so that we only consider smokers (we could just as easily restrict on non-smokers and this would work too)

In [ ]:
data2 = data[data['smoking'] == 0]

After we do this, the association between coffee and lung cancer risk vanishes entirely!

In [ ]:
clean_corr(
    pearsonr(data2['coffee'], data2['lung cancer risk'])
)

In [ ]:
plt.hexbin(data2['coffee'], data2['lung cancer risk'], gridsize=50)

## Collider

Now let's consider a collider causal relationship. Controlling for a collider can induce a false association between two variables if you control / stratify on it. For the following exercise, there shouldn't be an association between your genetic risk of diabetes and your family's household income when you were a child.

In [ ]:
cie_example = StructuralCausalModel({
    "your genetic risk of diabetes": lambda n_samples: np.random.normal(loc=1, scale=0.5, size=n_samples), 
    "household income during your childhood": lambda n_samples: np.random.normal(loc=3, scale=1, size=n_samples), 
    "mother's diabetes": logistic_model(["your genetic risk of diabetes", "household income during your childhood"], [1.5,-0.5])
})

In [ ]:
cie_cgm = cie_example.cgm
cie_cgm.draw()

In [ ]:
data = cie_example.sample(n_samples=100000)
data.head()

<div class="alert alert-success">
    <h3>EXERCISE: determine the raw association between genetic risk of diabetes and household income during childhood</h3>
</div>

In [ ]:
clean_corr(
    pearsonr(______, ______)
)

Let's restrict the dataset to only those with high numbers of sick days taken. We've now stratified, controlled for a collider... bad idea.

In [ ]:
data2 = data[data["mother's diabetes"] == 1]

<div class="alert alert-success">
    <h3>EXERCISE: Is there still zero association between genetic risk of diabetes and household income during childhood?</h3>
</div>

In [ ]:
clean_corr(
    pearsonr(______, ______)
)

## Mediation

<div class="alert alert-success">
    <h3>EXERCISE: Create a toy model demonstrating the causal relationship of "mediation". Here's a tip, in three variables you could show that smoking leads to clinical signs of lung damage, which in turn leads to lung cancer</h3>
</div>

In [ ]:
cie_example = StructuralCausalModel({
    "______": ______, # Tip: pearson correlations only work between two continuous variables, so in this toy example, let's make smoking and lung cancer continuous...
    "signs of lung damage": logistic_model(["smoking"], [5]),
    "lung cancer": linear_model(["signs of lung damage"], [5]),
})

<div class="alert alert-success">
    <h3>EXERCISE: Can you demonstrate that there is a relationship between the two outer variables (that are mediated by a third variable)?</h3>
</div>

In [ ]:
clean_corr(
    pearsonr(______, ______)
)

<div class="alert alert-success">
    <h3>EXERCISE: Try restricting the mediator in this dataset. Basically, filter the dataset so there is no variability in the mediator, and then you have broken the relationship between the two outer variables!</h3>
</div>

<div class="alert alert-success">
    <h3>EXERCISE: Can you confirm that the relationship between smoking and cancer has been broken?</h3>
</div>

## Unrelated predictors

<div class="alert alert-success">
    <h3>EXERCISE: Are you following the basic workflow we've been using above? Try to replicate it to show the causal relationship of an "unrelated predictor". This is a variable that causes some outcome, but is completely unrelated to a variable you're interested in. Because of this, stratifying / controlling for this unrelated predictor doesn't alter the relationship between your variable of interest and the outcome.</h3>
</div>